In [1]:
##Importando os pacotes e módulos necessários
import pandas as pd
import numpy as np
import math

#Para Regressão Linear Simples e Teste F
from scipy import stats
#Para Regressão Linear Múltipla (OLS, GLS e WLS) e Testes Estatísticos
import statsmodels.api as sm
#Para Regressão em Painel
from linearmodels import PanelOLS, FirstDifferenceOLS, PooledOLS

#Pacotes para gráficos (caso precise)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Pacotes para fazer a coleta dos dados armazenados no mesmo diretório e outros pacotes gerais
import os
import pathlib
import glob
from IPython.display import clear_output

####################################### Criando as Funções ###############################################################

def coletar_dados(nome = ""):
    '''
    Função que le os arquivos do Stata (.dta) - NÃO COLOQUE A EXTENSÃO NA HORA DE NOMEAR O "NOME"!
    O arquivo deve estar na mesma pasta do arquivo de Python ou do notebook do jupyter.
    Deixe em branco para ler o arquivo mais recentemente adicionado à pasta.
    '''

    global df

    #Pegando qual a pasta do arquivo que está sendo usado pra programar
    caminho = pathlib.Path().absolute()

    #No meu caso específico:
    caminho_vinicius = f"{caminho}/datasets"

    #checando se o nome foi inserido ou não; caso não, pegar o arquivo .dta mais recente
    if nome == "":
        try:
            arquivo = max(glob.glob(f"{str(caminho)}/*.dta"), key=os.path.getctime)
            df = pd.read_stata(arquivo)
            print(f"{arquivo}.dta foi lido com sucesso!")
            return df
        except:
            arquivo = max(glob.glob(f"{str(caminho_vinicius)}/*.dta"), key=os.path.getctime)
            df = pd.read_stata(arquivo)
            print(f"{arquivo}.dta foi lido com sucesso!")
            return df
    else:
        try:
            arquivo = f"{str(caminho)}/{str(nome)}.dta"
            df = pd.read_stata(arquivo)
            print(f"{nome}.dta foi lido com sucesso!")
            return df
        except:
            try:
                arquivo = f"{str(caminho_vinicius)}/{str(nome)}.dta"
                df = pd.read_stata(arquivo)
                print(f"{nome}.dta foi lido com sucesso!")
                return df
            except: #caso não tenha sido encontrado o arquivo com o nome inserido
                print('''
                Não foi possível achar o arquivo :(\n
                Verifique se seu nome está correto (sem a extensão) e se ele está no mesmo diretório do programa!
                ''')

def Regressão_Simples(Lista_x, Lista_y):
    '''
    Função que calcula as estatísticas de uma regressão simples

    Lista_x: lista com os valores de x;
    Lista_y: lista com os valores de y;
    '''

    global Lista_ychapeu_simples, Resíduos_simples
    
    #calculando o número de observações e as médias
    Número_de_Observações = len(Lista_x)
    Média_x = sum(Lista_x)/Número_de_Observações
    Média_y = sum(Lista_y)/Número_de_Observações

    #Calculando os coeficientes do modelo
    B1, B0, R, valor_P, DP = stats.linregress(Lista_x, Lista_y)
    #Gerando os valores previstos
    Lista_ychapeu_simples = [round(B0 + B1 * i,3) for i in Lista_x]
    #Calculando os Resíduos
    Resíduos_simples = [(j - k) for j,k in zip(Lista_y, Lista_ychapeu_simples)]
    
    #Calculando R-quadrados e a Soma dos Quadrados das Variáveis
    R_quadrado = round(R**2,5)
    SQTx = sum([(i - Média_x)**2 for i in Lista_x])
    SQTy = sum([(i - Média_y)**2 for i in Lista_y])
    SQEy = sum([(i - Média_y)**2 for i in Lista_ychapeu_simples])
    SQR = sum([i**2 for i in Resíduos_simples]) 

    #Calculando a Variância da Regressão e dos  Coeficientes
    VarianciaReg = SQR/(Número_de_Observações - 2)
    EPR = math.sqrt(VarianciaReg)
    VarB1 = VarianciaReg/SQTx
    VarB0 = (VarianciaReg * sum([i**2 for i in Lista_x]))/(Número_de_Observações * SQTx)
    
    #Calculando da estatistica t com intervalo de confiança de 95%  (p/ gerar os intervalos de confiança dos estimadores)
    Estatistica_t_Critica = stats.t.ppf(0.95, Número_de_Observações - 2)

    #Calculando os estimadores do limite inferior e superior
    B1_inferior = B1 - math.sqrt(VarB1) * Estatistica_t_Critica
    B1_superior = B1 + math.sqrt(VarB1) * Estatistica_t_Critica
    B0_inferior = B0 - math.sqrt(VarB0) * Estatistica_t_Critica
    B0_superior = B0 + math.sqrt(VarB0) * Estatistica_t_Critica
    
    #Gerando o Relatório
    Relatório = f'''
    Número de Observações = {Número_de_Observações}\n
    B0 = {round(B0,5)}\t B1 = {round(B1,5)}\t R-quadrado = {R_quadrado}\n
    Estimador da Variância = {round(VarianciaReg,5)}\t Erro Padrão da Regressão = {round(EPR,5)}\n
    Variância de B1 = {round(VarB1,5)}\t Variância de B0 = {round(VarB0,5)}\n
    Intervalo de Confiança de 95% para B1 (Inferior; B1; Superior): {round(B1_inferior,4)}; {round(B1,4)}; {round(B1_superior,4)}\n
    Intervalo de Confiança de 95% para B0 (Inferior; B0; Superior): {round(B0_inferior,4)}; {round(B0,4)}; {round(B0_superior,4)}\n
    Para ver os valores previstos, basta chamar a variável 'Lista_ychapeu_simples'\n
    Para ver os resíduos, chame a variável 'Resíduos_simples'
    '''
    print (Relatório)
    
    ##Criando um gráfico
    #Deixando o estilo bonitinho
    sns.set_style(style="white")

    #Criando o objeto gráfico
    Grafico = sns.regplot(x = Lista_x, y = Lista_y, scatter_kws={"color": "black"}, line_kws={"color": "red"})
    Grafico.set_title("Resultado da Regressão",fontsize = 11)
    plt.show()

def Regressão_Múltipla(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla, sendo, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares (OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x

    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = sm.OLS(y,X)

    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'HC1', use_t = True)
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(VarianciaReg)
    
    ##Printando o Resultado
    #print('Parâmetros:\n', Resultado.params) #O primeiro resultado equivale ao intercepto
    #print('\nDesvios Padrões:\n', Resultado.bse)
    #print('Valores Previstos: ', Resultado.predict())
    #print('\nR2:', Resultado.rsquared)
    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}\n")
    print(Resultado.summary())

    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
    print("""
    Valores de condição maiores que 20 indicam problemas de multicolinearidade.
    Para ver como achar esse número, entre em https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html"""
    )

def Regressão_MQP(x, y, pesos, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla usando mínimos quadrados ponderados, ou seja,
    recomendada quando o erro é heteroscedástico E se sabe a função da constante. Ela é, por default, computada com um intercepto e com erros padrões não robustos.
    multiplicativa da variância do erro.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    pesos: 1/h, sendo h a constante multiplicativa da variância do erro;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares(OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = sm.WLS(y,X, weights = pesos)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'HC1', use_t = True)
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(VarianciaReg)
    
    ##Printando o Resultado
    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}\n")
    print(Resultado.summary())

    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
    print("""
    Valores de condição maiores que 20 indicam problemas de multicolinearidade
    Para ver como achar esse número, entre em https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html"""
    )
    
def Regressão_MQGF(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla usando mínimos quadrados generalizados factíveis, ou seja,
    recomendada quando o erro é heteroscedástico E NÃO se sabe a função da constante multiplicativa da variância do erro, sendo os pesos estimados
    regridindo o log dos quadrados dos resíduos sobre as variáveis explicativas. Os estimadores MQP são gerados com o peso estimado.
    Ela é, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #Regredindo os valores normalmente a fim de pegar os resíduos
    Regressão_Múltipla(x,y, constante, robusta)
    clear_output()

    #Coletando o log dos quadrados dos resíduos
    Log_Res_Quad = np.log(Resíduos**2)

    #Regredindo Log_Res_Quad sobre as variáveis explicativas
    Regressão_Múltipla(x,Log_Res_Quad, constante, robusta)
    clear_output()

    #Estimando os pesos
    Pesos = np.exp(Lista_ychapeu)

    #Fazendo uma Regressão MQP
    Regressão_MQP(x,y, 1/Pesos, constante, robusta)

def Teste_LM(x, y, Restrições, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste LM e dá o resultado teste de hipótese para o caso de todas as restrições serem conjuntamente estatisticamente não-significantes.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Restrições: lista ou array com os valores a serem tirados do modelo restrito;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''

    ##Definindo as variáveis de cada modelo
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)
    
    #Fazendo a regressão do modelo restrito e armazenando os resíduos
    Regressão_Múltipla(df[ModeloRestrito], y)
    Resíduos_r = Resíduos

    #Fazendo a regressão dos resíduos sobre as variáveis independentes e armazenando o R2
    Regressão_Múltipla(x, Resíduos_r)
    Ru = Resultado.rsquared

    #Calculando a estatística LM
    LM = Numero_de_Observações*Ru

    #Calculando o p-valor
    ##Calculando o P-valor de F
    P_valor = stats.chi2.sf(LM,GL_r)

    #Limpando a tela
    clear_output()

    #Printando o resultado
    if Nivel_de_Significância > P_valor:
        print(f"O valor de LM é {round(LM,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de LM é {round(LM,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho a um nível de significância de {Nivel_de_Significância*100}%.")


def Teste_F(x, y, Restrições, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste F e dá o resultado teste de hipótese para o caso de todas as restrições serem conjuntamente estatisticamente não-significantes.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Restrições: lista ou array com os valores a serem tirados do modelo restrito;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''

    ##Definindo as variáveis de cada modelo
    #para testar igualdade dos coeficientes, F2, p_valueF2 = results.Ftest(['ACT', 'skipped'], equal=True)
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)

    ##Fazendo as regressões de cada modelo
    Regressão_Múltipla(x, y)
    SQR_ir = SQR
    VarianciaReg_ir = EPR**2

    Regressão_Múltipla(df[ModeloRestrito], y)
    SQR_r = SQR

    #Limpando a tela
    clear_output()
    
    ##Calculando F
    F = (SQR_r - SQR_ir)/(len(Restrições)*VarianciaReg_ir)

    ##Calculando o P-valor de F
    P_valor = stats.f.sf(F,GL_r,GL_ir)

    if Nivel_de_Significância > P_valor:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho a um nível de significância de {Nivel_de_Significância*100}%.")

def Teste_t_Dois_Coeficientes_Iguais(x, y, Coeficientes_Testados_para_serem_iguais, Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste t para verificar se dois coeficientes são iguais.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Coeficientes_Testados_para_serem_iguais: array com os valores dos coeficientes que querem ser testados;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    
    ##Fazendo a regressão do modelo irrestrito
    Regressão_Múltipla(x, y)
    clear_output()

    #Fazendo o objeto de lista que será usado no teste
    Teste =[0]
    Num_de_Variaveis = 1

    for i in list(x):
        if i not in list(Coeficientes_Testados_para_serem_iguais):
            Teste.append(0)
        elif (Num_de_Variaveis % 2 == 0):
            Teste.append(-1)
        else:
            Teste.append(1)
            Num_de_Variaveis += 1

    Teste_t = Resultado.t_test(Teste)
    print(f"A estatística do teste é {np.around(Teste_t.tvalue[0],3)}, o que resulta em um p-valor de {np.around(Teste_t.pvalue[0],6)}")

def Teste_Heteroscedasticidade_BP(x, y, constante = "S", Nivel_de_Significância = 0.05, Estatística = "LM"):
    '''
    Função que executa o teste de Breusch-Pagan para a heteroscedasticidade.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    Estatística = LM ou F
    '''

    #Fazendo a regressão e limpando a tela
    Regressão_Múltipla(x,y,constante)
    clear_output()

    #Calculando o quadrado dos resíduos
    Res_Quad = Resíduos**2

    #Realizando o teste F ou LM de Res_Quad sobre as variaveis dependentes para ver se há correlação
    if Estatística == "LM":
        Teste_LM(x, Res_Quad, x, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")
    else:
        Teste_F(x, Res_Quad, x, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")

def Teste_Heteroscedasticidade_White(x, y, constante = "S", Nivel_de_Significância = 0.05, Estatística = "LM"):
    '''
    Função que executa o teste de White (modificado por Wooldridge) para a heteroscedasticidade.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    Estatística: LM ou F
    '''

    #Fazendo a regressão e limpando a tela
    Regressão_Múltipla(x,y,constante)
    clear_output()

    #Calculando o quadrado dos resíduos
    Res_Quad = Resultado.resid**2

    #Calculando o quadrado dos valores previstos
    Previstos = Lista_ychapeu
    Previstos2 = Previstos**2

    #Criando um dataframe pra armazenar esses valores
    dfy_y2 = pd.DataFrame({"y":Previstos,"y2":Previstos2})
    y_y2 = dfy_y2[['y','y2']]

    #Realizando o teste F ou LM de Res_Quad sobre y e y^2
    if Estatística == "LM":
        Teste_LM(y_y2, Res_Quad, y_y2, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")
    else:
        Teste_F(y_y2, Res_Quad, y_y2, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")

def RESET(x, y, constante = "S", robusta = "N", Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste RESET para verificar a adequação das formas funcionais.
    Ho: o modelo está bem especificado.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    #Fazendo uma regressão múltipla e limpando a tela
    Regressão_Múltipla(x, y, constante)
    clear_output()

    #Verificando o tipo da covariância selecionada
    if robusta == "N":
        tipo = 'nonrobust'
    else:
        tipo = 'HC1'

    Teste = sm.stats.diagnostic.linear_reset(Resultado, power = 2, use_f = False, cov_type = tipo)
    
    if Teste.pvalue < Nivel_de_Significância:
        print(f"""
        O p-valor do teste foi de {np.around(Teste.pvalue,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, rejeita-se Ho (o modelo está MAL especificado)."""
        )
    else:
        print(f"""
        O p-valor do teste foi de {np.around(Teste.pvalue,6)}, maior que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, não se rejeita Ho (o modelo NÃO está MAL especificado)"""
        )

def Teste_J_Davidson_MacKinnon(x1,x2, y, constante = "S", robusta = "N", Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste J para verificar qual o modelo mais adequado (dentre os dois colocados).
    Ho: o modelo 1 é preferível (ver o p-valor do último coeficiente).

    x1: lista ou array com os valores das variáveis independentes do primeiro modelo;
    x2: lista ou array com os valores das variáveis independentes do segundo modelo;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    
    #Fazendo a regressão do segundo modelo
    Regressão_Múltipla(x2, y, constante, robusta)
    clear_output()

    #Criando um novo dataframe e adicionando os valores previstos do modelo 2 à x
    Valores_Previstos_2 = pd.DataFrame({'Previsão M1':Lista_ychapeu})
    x = pd.concat([x1, Valores_Previstos_2], axis=1, sort=False)

    #Fazendo a regressão do primeiro modelo sobre x
    Regressão_Múltipla(x, y, constante, robusta)
    clear_output()

    #Pegando o p-valor do teste
    P_valor = Resultado.pvalues[-1]

    if P_valor < Nivel_de_Significância:
        print(f"""
        O p-valor do teste foi de {np.around(P_valor,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, rejeita-se Ho (ou seja, o modelo 2 ({list(x2)}) é mais bem especificado)."""
        )
    else:
        print(f"""
        O p-valor do teste foi de {np.around(P_valor,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, não se rejeita Ho (ou seja, o modelo 1 ({list(x1)}) é mais bem especificado)."""
        )

######### Funções de Dados em Painel #########
def Arrumar_Painel():
    '''
    Função que transforma o painel num formato que o PanelOLS consegue ler (index multinível e coluna do tipo categoria para os anos)
    '''
    global df

    # pedir a coluna com os indivíduos; se o nome for inválido, sair da função.
    coluna_individuos = str(input('Qual o rótulo da coluna de indivíduos/observações?\n'))
    if coluna_individuos not in df.columns:
        print("Coluna de indivíduos não está no dataframe. Insira uma coluna válida e tente novamente!")
        return None
    
    # pedir a coluna com os períodos de tempo; se o valor for inválido, sair da função.
    coluna_tempo = str(input('Qual o rótulo da coluna de tempo?\n'))
    if coluna_tempo not in df.columns:
        print("Coluna de tempo não está no dataframe. Insira uma coluna válida e tente novamente!")
        return None

    ## arrumando o painel
    periodos = pd.Categorical(df[coluna_tempo])
    df = df.set_index([coluna_individuos,coluna_tempo])
    df[coluna_tempo] = periodos
    return df

def Reg_Primeiras_Diferenças (x,y, robusta = "N"):
    '''
    Função que calcula uma regressão de primeiras diferenças SEM um intercepto, sendo, por default, computada com erros padrões não robustos.
    Para calcular a regressão com um intercepto, ver o notebook "Cap 13 e 14".
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''
    Modelo = FirstDifferenceOLS(y, x)
    Resultado = Modelo.fit()
    print(Resultado)

def Regressão_Efeitos_Fixos(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão de efeitos fixos, sendo, por default, computada com um intercepto e com erros padrões não robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = PanelOLS(y,X, entity_effects=True)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'robust')
    print(Resultado)

## Primeiras Diferenças

### Exemplo 13.8 - Pg. 442

In [3]:
pathlib.Path().absolute()

PosixPath('/Users/vinicius/Meu Drive/UnB/Econometria')

In [7]:
## Lendo o arquivo .dta
coletar_dados("ezunem")


                Não foi possível achar o arquivo :(

                Verifique se seu nome está correto (sem a extensão) e se ele está no mesmo diretório do programa!
                


In [104]:
## mostrando o dataframe
display(df)

,year,uclms,ez,d81,d82,d83,d84,d85,d86,d87,...,c14,c15,c16,c17,c18,c19,c20,c21,c22,city
0,1980,166746,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1981,83561,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1982,158146,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1983,83572,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1984,45949,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,1984,80605,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,22
194,1985,82758,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,22
195,1986,67815,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,22
196,1987,67762,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,22


In [105]:
## temos que calcular o log de uclms, então:
df['luclms'] = np.log(df['uclms'])

In [106]:
## temos que calcular a diferença de duas variaveis:
#luclms e ez (dummy que diz se a cidade i no tempo t possuia uma zona industrial)
df['cluclms'] = df.luclms.diff()
df['cez'] = df.ez.diff()

# ATENCAO: não calcular para o ano de 1980!
df_81_88 = df.loc[df['year'] != 1980]
display(df_81_88)

,year,uclms,ez,d81,d82,d83,d84,d85,d86,d87,...,c17,c18,c19,c20,c21,c22,city,luclms,cluclms,cez
1,1981,83561,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,11.333332,-0.690895,0.0
2,1982,158146,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,11.971274,0.637942,0.0
3,1983,83572,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,11.333464,-0.637810,0.0
4,1984,45949,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,10.735287,-0.598176,1.0
5,1985,48848,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,10.796469,0.061181,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,1984,80605,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,22,11.297316,-0.547270,0.0
194,1985,82758,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,22,11.323676,0.026360,0.0
195,1986,67815,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,22,11.124539,-0.199137,0.0
196,1987,67762,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,22,11.123757,-0.000782,0.0


In [107]:
## Fazendo a regressao multipla
y = df_81_88['cluclms']
x = df_81_88[['d82','d83','d84','d85','d86','d87','d88','cez']]
Regressão_Múltipla(x,y)

O erro padrão da regressão é 0.21606 e a SQR é 7.79584

                            OLS Regression Results                            
Dep. Variable:                cluclms   R-squared:                       0.623
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     34.50
Date:                Tue, 16 Feb 2021   Prob (F-statistic):           1.08e-31
Time:                        16:06:19   Log-Likelihood:                 24.554
No. Observations:                 176   AIC:                            -31.11
Df Residuals:                     167   BIC:                            -2.573
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

## Efeitos Fixos

### Exemplo 14.2 - Pg. 453

In [8]:
coletar_dados("wagepan")

wagepan.dta foi lido com sucesso!


,nr,year,agric,black,bus,construc,ent,exper,fin,hisp,...,union,lwage,d81,d82,d83,d84,d85,d86,d87,expersq
0,13,1980,0,0,1,0,0,1,0,0,...,0,1.197540,0,0,0,0,0,0,0,1
1,13,1981,0,0,0,0,0,2,0,0,...,1,1.853060,1,0,0,0,0,0,0,4
2,13,1982,0,0,1,0,0,3,0,0,...,0,1.344462,0,1,0,0,0,0,0,9
3,13,1983,0,0,1,0,0,4,0,0,...,0,1.433213,0,0,1,0,0,0,0,16
4,13,1984,0,0,0,0,0,5,0,0,...,0,1.568125,0,0,0,1,0,0,0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4355,12548,1983,0,0,0,1,0,8,0,0,...,0,1.591879,0,0,1,0,0,0,0,64
4356,12548,1984,0,0,0,1,0,9,0,0,...,1,1.212543,0,0,0,1,0,0,0,81
4357,12548,1985,0,0,0,1,0,10,0,0,...,0,1.765962,0,0,0,0,1,0,0,100
4358,12548,1986,0,0,0,0,0,11,0,0,...,1,1.745894,0,0,0,0,0,1,0,121


In [9]:
# criando os termos de interação entre as dummies anuais e o educ
df['d81_educ'] = df['d81']*df['educ']
df['d82_educ'] = df['d82']*df['educ']
df['d83_educ'] = df['d83']*df['educ']
df['d84_educ'] = df['d84']*df['educ']
df['d85_educ'] = df['d85']*df['educ']
df['d86_educ'] = df['d86']*df['educ']
df['d87_educ'] = df['d87']*df['educ']

In [5]:
def Arrumar_Painel():
    '''
    Função que transforma o painel num formato que o PanelOLS consegue ler (index multinível e coluna do tipo categoria para os anos)
    '''
    global df

    # pedir a coluna com os indivíduos; se o nome for inválido, sair da função.
    coluna_individuos = str(input('Qual o rótulo da coluna de indivíduos/observações?\n'))
    if coluna_individuos not in df.columns:
        print("Coluna de indivíduos não está no dataframe. Insira uma coluna válida e tente novamente!")
        return None
    
    # pedir a coluna com os períodos de tempo; se o valor for inválido, sair da função.
    coluna_tempo = str(input('Qual o rótulo da coluna de tempo?\n'))
    if coluna_tempo not in df.columns:
        print("Coluna de tempo não está no dataframe. Insira uma coluna válida e tente novamente!")
        return None

    ## arrumando o painel
    periodos = pd.Categorical(df[coluna_tempo])
    df = df.set_index([coluna_individuos,coluna_tempo])
    df[coluna_tempo] = periodos
    display(df)

In [6]:
Arrumar_Painel()

agric  black  bus  construc  ent  exper  fin  hisp  poorhlth  \
nr    year                                                                 
13    1980      0      0    1         0    0      1    0     0         0   
      1981      0      0    0         0    0      2    0     0         0   
      1982      0      0    1         0    0      3    0     0         0   
      1983      0      0    1         0    0      4    0     0         0   
      1984      0      0    0         0    0      5    0     0         0   
...           ...    ...  ...       ...  ...    ...  ...   ...       ...   
12548 1983      0      0    0         1    0      8    0     0         0   
      1984      0      0    0         1    0      9    0     0         0   
      1985      0      0    0         1    0     10    0     0         0   
      1986      0      0    0         0    0     11    0     0         0   
      1987      0      0    0         0    0     12    0     0         0   

            hours  ...  d87  expersq  d81_educ  d82_educ  d83_educ  d84_educ  \
nr    year         ...                                                         
13    1980   2672  ...    0        1         0         0         0         0   
      1981   2320  ...    0        4        14         0         0         0   
      1982   2940  ...    0        9         0        14         0         0   
      1983   2960  ...    0       16         0         0        14         0   
      1984   3071  ...    0       25         0         0         0        14   
...           ...  ...  ...      ...       ...       ...       ...       ...   
12548 1983   2080  ...    0       64         0         0         9         0   
      1984   2080  ...    0       81         0         0         0         9   
      1985   2080  ...    0      100         0         0         0         0   
      1986   2080  ...    0      121         0         0         0         0   
      1987   3380  ...    1      144         0         0         0         0   

            d85_educ  d86_educ  d87_educ  year  
nr    year                                      
13    1980         0         0         0  1980  
      1981         0         0         0  1981  
      1982         0         0         0  1982  
      1983         0         0         0  1983  
      1984         0         0         0  1984  
...              ...       ...       ...   ...  
12548 1983         0         0         0  1983  
      1984         0         0         0  1984  
      1985         9         0         0  1985  
      1986         0         9         0  1986  
      1987         0         0         9  1987  

[4360 rows x 50 columns]

In [7]:
x = df[['married','union','d81_educ','d82_educ','d83_educ','d84_educ','d85_educ','d86_educ','d87_educ','year']]
y = df['lwage']

In [8]:
from linearmodels import PanelOLS, FirstDifferenceOLS

def Regressão_Efeitos_Fixos(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão de efeitos fixos, sendo, por default, computada com um intercepto e com erros padrões não robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = PanelOLS(y,X, entity_effects=True)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'robust')
    
    print(Resultado)

In [9]:
Regressão_Efeitos_Fixos(x,y, robusta = "N")

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1708
Estimator:                   PanelOLS   R-squared (Between):              0.0905
No. Observations:                4360   R-squared (Within):               0.1708
Date:                Sat, Jun 05 2021   R-squared (Overall):              0.1277
Time:                        13:53:29   Log-likelihood                   -1350.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      48.907
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(16,3799)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             48.907
                            

In [10]:
def fixed_effects(panel_data, formula, time_effects = False, cov = "unadjusted"):
    """
    Fits a standard Fixed Effeects model with the corresponding covariance matrix.
    It can be estimated WITH and WITHOUT a constant.
    It is preferred when the unobserved effects are correlated with the error term
    and, therefore, CAN'T estimate constant terms.

    panel_data : dataframe (which must be in a panel structure)
    formula : patsy formula
    time_effects : bool, default to False
        Whether to include time effects alongside entity effects
    cov : str
        unadjusted: common standard errors
        robust: robust standard errors
        kernel: robust to heteroskedacity AND serial autocorrelation
        cluster or clustered: clustered standard errors by the entity column
    """

    ## creating model instance
    if time_effects:
        formula += ' + EntityEffects + TimeEffects'
    else:
        formula += ' + EntityEffects'

    mod = PanelOLS.from_formula(formula = formula, data = panel_data, drop_absorbed=True)

    if cov == 'clustered' or cov == 'cluster':
        mod = mod.fit(cov_type = 'clustered', cluster_entity = True)
    else:
        mod = mod.fit(cov_type = cov)
    
    print(mod.summary)
    return mod

In [14]:
formula = 'lwage ~ 1 + d81 + d82 + d83 + d84 + d85 + d86 + d87 + d81:educ + d82:educ + d83:educ + d84:educ + d85:educ + d86:educ + d87:educ + union + married'
mod = fixed_effects(df, formula)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1708
Estimator:                   PanelOLS   R-squared (Between):              0.0905
No. Observations:                4360   R-squared (Within):               0.1708
Date:                Sat, Jun 05 2021   R-squared (Overall):              0.1277
Time:                        13:56:19   Log-likelihood                   -1350.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      48.907
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(16,3799)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             48.907
                            

## Efeitos Aleatórios e MQO Agrupado

(usado quando não há correlação entre as características não observadas e todas as variáveis explicativas)

### Exemplo 14.4 - Pg. 460

Continuaremos usando o arquivo WAGEPAN.DTA, que já foi arrumado pelas funções do exemplo acima.

O objetivo é estimar e comparar 3 equações: uma por MQO Agrupado, outra por EF e outra por EA.

Contudo, usaremos como variáveis dependentes "educ", "black", "hisp", "exper", "expersq","married" e "union". Como EF só admite variáveis exógenas que variam de forma não uniforme ao longo do tempo, seu vetor x será "expersq","married" e "union". Todas as regressões tem um conjunto de dummies temporais completo ("year").

In [119]:
#estimando por EF
x = df[["expersq","married","union","year"]]
y = df['lwage']
Regressão_Efeitos_Fixos(x,y, robusta = "N")

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:                   PanelOLS   R-squared (Between):             -0.0052
No. Observations:                4360   R-squared (Within):               0.1806
Date:                Tue, Feb 16 2021   R-squared (Overall):              0.0807
Time:                        16:29:06   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      83.851
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             83.851
                            

In [123]:
from linearmodels import PooledOLS, RandomEffects

In [126]:
# estimando por MQO agrupado
x = df[["educ", "black", "hisp", "exper", "expersq","married","union","year"]]
X = sm.add_constant(x)
Modelo = PooledOLS(y,X)
Resultado = Modelo.fit()
print(Resultado)

                          PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.1893
Estimator:                  PooledOLS   R-squared (Between):              0.2066
No. Observations:                4360   R-squared (Within):               0.1692
Date:                Tue, Feb 16 2021   R-squared (Overall):              0.1893
Time:                        16:38:08   Log-likelihood                   -2982.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      72.459
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             72.459
                            

In [124]:
# estimando por efeitos aleatórios
Modelo = RandomEffects(y,X)
Resultado = Modelo.fit()
print(Resultado)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:              RandomEffects   R-squared (Between):              0.1853
No. Observations:                4360   R-squared (Within):               0.1799
Date:                Tue, Feb 16 2021   R-squared (Overall):              0.1828
Time:                        16:33:24   Log-likelihood                   -1622.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      68.409
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             68.409
                            

In [127]:
def Reg_Painel_MQO_Agrupado(x, y, constante = "S", robusta = "S"):
    '''
    Função que calcula uma regressão por MQO agrupado, sendo, por default, computada com um intercepto e com erros padrões  robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão robustos.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = PooledOLS(y,X)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'robust')
    print(Resultado)

def Reg_Painel_Efeitos_Aleatórios(x, y, constante = "S"):
    '''
    Função que calcula uma regressão de efeitos fixos, sendo, por default, computada com um intercepto e com erros padrões  robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão robustos.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo
    Modelo = RandomEffects(y,X)
    Resultado = Modelo.fit()
    print(Resultado)

In [2]:
##Importando os pacotes e módulos necessários
import pandas as pd
import numpy as np
import math

#Para Regressão Linear Simples e Teste F
from scipy import stats
#Para Regressão Linear Múltipla (OLS, GLS e WLS) e Testes Estatísticos
import statsmodels.api as sm
import econtools
import econtools.metrics as mt
#Para Regressão em Painel
from linearmodels import PanelOLS, FirstDifferenceOLS, PooledOLS, RandomEffects

#Pacotes para gráficos (caso precise)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

#Pacotes para fazer a coleta dos dados armazenados no mesmo diretório e outros pacotes gerais
import os
import pathlib
import glob
from IPython.display import clear_output
import gc

####################################### Criando as Funções ###############################################################

def coletar_dados(nome = ""):
    '''
    Função que le os arquivos do Stata (.dta) - NÃO COLOQUE A EXTENSÃO NA HORA DE NOMEAR O "NOME"!
    O arquivo deve estar na mesma pasta do arquivo de Python ou do notebook do jupyter.
    Deixe em branco para ler o arquivo mais recentemente adicionado à pasta.
    '''

    global df

    #Pegando qual a pasta do arquivo que está sendo usado pra programar
    caminho = pathlib.Path().absolute()

    #No meu caso específico:
    caminho_vinicius = f"{caminho}/datasets"

    #checando se o nome foi inserido ou não; caso não, pegar o arquivo .dta mais recente
    if nome == "":
        try:
            arquivo = max(glob.glob(f"{str(caminho)}/*.dta"), key=os.path.getctime)
            df = pd.read_stata(arquivo)
            print(f"{arquivo}.dta foi lido com sucesso!")
            return df
        except:
            arquivo = max(glob.glob(f"{str(caminho_vinicius)}/*.dta"), key=os.path.getctime)
            df = pd.read_stata(arquivo)
            print(f"{arquivo}.dta foi lido com sucesso!")
            return df
    else:
        try:
            arquivo = f"{str(caminho)}/{str(nome)}.dta"
            df = pd.read_stata(arquivo)
            print(f"{nome}.dta foi lido com sucesso!")
            return df
        except:
            try:
                arquivo = f"{str(caminho_vinicius)}/{str(nome)}.dta"
                df = pd.read_stata(arquivo)
                print(f"{nome}.dta foi lido com sucesso!")
                return df
            except: #caso não tenha sido encontrado o arquivo com o nome inserido
                print('''
                Não foi possível achar o arquivo :(\n
                Verifique se seu nome está correto (sem a extensão) e se ele está no mesmo diretório do programa!
                ''')

def Regressao_Multipla(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla, sendo, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares (OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x

    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = sm.OLS(y,X)

    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'HC1', use_t = True)
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(VarianciaReg)
    
    ##Printando o Resultado
    print(Resultado.summary())

    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}")
    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
    print("""
    Valores de condição maiores que 20 indicam problemas de multicolinearidade.
    Para ver como achar esse número, entre em https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html"""
    )

def Regressao_MQP(x, y, pesos, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla usando mínimos quadrados ponderados, ou seja,
    recomendada quando o erro é heteroscedástico E se sabe a função da constante. Ela é, por default, computada com um intercepto e com erros padrões não robustos.
    multiplicativa da variância do erro.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    pesos: 1/h, sendo h a constante multiplicativa da variância do erro;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares(OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = sm.WLS(y,X, weights = pesos)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'HC1', use_t = True)
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(VarianciaReg)
    
    ##Printando o Resultado
    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}\n")
    print(Resultado.summary())

    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
    print("""
    Valores de condição maiores que 20 indicam problemas de multicolinearidade
    Para ver como achar esse número, entre em https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html"""
    )
    
def Regressao_MQGF(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla usando mínimos quadrados generalizados factíveis, ou seja,
    recomendada quando o erro é heteroscedástico E NÃO se sabe a função da constante multiplicativa da variância do erro, sendo os pesos estimados
    regridindo o log dos quadrados dos resíduos sobre as variáveis explicativas. Os estimadores MQP são gerados com o peso estimado.
    Ela é, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #Regredindo os valores normalmente a fim de pegar os resíduos
    Regressao_Multipla(x,y, constante, robusta)
    clear_output()

    #Coletando o log dos quadrados dos resíduos
    Log_Res_Quad = np.log(Resíduos**2)

    #Regredindo Log_Res_Quad sobre as variáveis explicativas
    Regressao_Multipla(x,Log_Res_Quad, constante, robusta)
    clear_output()

    #Estimando os pesos
    Pesos = np.exp(Lista_ychapeu)

    #Fazendo uma Regressão MQP
    Regressao_MQP(x,y, 1/Pesos, constante, robusta)

def Teste_LM(x, y, Restrições, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste LM e dá o resultado teste de hipótese para o caso de todas as restrições serem conjuntamente estatisticamente não-significantes.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Restrições: lista ou array com os valores a serem tirados do modelo restrito;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''

    ##Definindo as variáveis de cada modelo
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)
    
    #Fazendo a regressão do modelo restrito e armazenando os resíduos
    Regressao_Multipla(df[ModeloRestrito], y)
    Resíduos_r = Resíduos

    #Fazendo a regressão dos resíduos sobre as variáveis independentes e armazenando o R2
    Regressao_Multipla(x, Resíduos_r)
    Ru = Resultado.rsquared

    #Calculando a estatística LM
    LM = Numero_de_Observações*Ru

    #Calculando o p-valor
    ##Calculando o P-valor de F
    P_valor = stats.chi2.sf(LM,GL_r)

    #Limpando a tela
    clear_output()

    #Printando o resultado
    if Nivel_de_Significância > P_valor:
        print(f"O valor de LM é {round(LM,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de LM é {round(LM,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho a um nível de significância de {Nivel_de_Significância*100}%.")


def Teste_F(x, y, Restrições, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste F e dá o resultado teste de hipótese para o caso de todas as restrições serem conjuntamente estatisticamente não-significantes.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Restrições: lista ou array com os valores a serem tirados do modelo restrito;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''

    ##Definindo as variáveis de cada modelo
    #para testar igualdade dos coeficientes, F2, p_valueF2 = results.Ftest(['ACT', 'skipped'], equal=True)
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)

    ##Fazendo as regressões de cada modelo
    Regressao_Multipla(x, y)
    SQR_ir = SQR
    VarianciaReg_ir = EPR**2

    Regressao_Multipla(df[ModeloRestrito], y)
    SQR_r = SQR

    #Limpando a tela
    clear_output()
    
    ##Calculando F
    F = (SQR_r - SQR_ir)/(len(Restrições)*VarianciaReg_ir)

    ##Calculando o P-valor de F
    P_valor = stats.f.sf(F,GL_r,GL_ir)

    if Nivel_de_Significância > P_valor:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho à significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho à significância de {Nivel_de_Significância*100}%.")

def Teste_t_Dois_Coeficientes_Iguais(x, y, Coeficientes_Testados_para_serem_iguais, Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste t para verificar se dois coeficientes são iguais.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Coeficientes_Testados_para_serem_iguais: array com os valores dos coeficientes que querem ser testados;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    
    ##Fazendo a regressão do modelo irrestrito
    Regressao_Multipla(x, y)
    clear_output()

    #Fazendo o objeto de lista que será usado no teste
    Teste =[0]
    Num_de_Variaveis = 1

    for i in list(x):
        if i not in list(Coeficientes_Testados_para_serem_iguais):
            Teste.append(0)
        elif (Num_de_Variaveis % 2 == 0):
            Teste.append(-1)
        else:
            Teste.append(1)
            Num_de_Variaveis += 1

    Teste_t = Resultado.t_test(Teste)
    print(f"A estatística do teste é {Teste_t.tvalue}, o que resulta em um p-valor bilateral de {Teste_t.pvalue} e em um p-valor unilateral de {Teste_t.pvalue/2}.")

def Teste_Heteroscedasticidade_BP(x, y, constante = "S", Nivel_de_Significância = 0.05, Estatística = "LM"):
    '''
    Função que executa o teste de Breusch-Pagan para a heteroscedasticidade.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    Estatística = LM ou F
    '''

    #Fazendo a regressão e limpando a tela
    Regressao_Multipla(x,y,constante)
    clear_output()

    #Calculando o quadrado dos resíduos
    Res_Quad = Resíduos**2

    #Realizando o teste F ou LM de Res_Quad sobre as variaveis dependentes para ver se há correlação
    if Estatística == "LM":
        Teste_LM(x, Res_Quad, x, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")
    else:
        Teste_F(x, Res_Quad, x, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")

def Teste_Heteroscedasticidade_White(x, y, constante = "S", Nivel_de_Significância = 0.05, Estatística = "LM"):
    '''
    Função que executa o teste de White (modificado por Wooldridge) para a heteroscedasticidade.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    Estatística: LM ou F
    '''

    #Fazendo a regressão e limpando a tela
    Regressao_Multipla(x,y,constante)
    clear_output()

    #Calculando o quadrado dos resíduos
    Res_Quad = Resultado.resid**2

    #Calculando o quadrado dos valores previstos
    Previstos = Lista_ychapeu
    Previstos2 = Previstos**2

    #Criando um dataframe pra armazenar esses valores
    dfy_y2 = pd.DataFrame({"y":Previstos,"y2":Previstos2})
    y_y2 = dfy_y2[['y','y2']]

    #Realizando o teste F ou LM de Res_Quad sobre y e y^2
    if Estatística == "LM":
        Teste_LM(y_y2, Res_Quad, y_y2, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")
    else:
        Teste_F(y_y2, Res_Quad, y_y2, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")

def RESET(x, y, constante = "S", robusta = "N", Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste RESET para verificar a adequação das formas funcionais.
    Ho: o modelo está bem especificado.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    #Fazendo uma regressão múltipla e limpando a tela
    Regressao_Multipla(x, y, constante)
    clear_output()

    #Verificando o tipo da covariância selecionada
    if robusta == "N":
        tipo = 'nonrobust'
    else:
        tipo = 'HC1'

    Teste = sm.stats.diagnostic.linear_reset(Resultado, power = 2, use_f = False, cov_type = tipo)
    
    if Teste.pvalue < Nivel_de_Significância:
        print(f"""
        O p-valor do teste foi de {np.around(Teste.pvalue,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, rejeita-se Ho (o modelo está MAL especificado)."""
        )
    else:
        print(f"""
        O p-valor do teste foi de {np.around(Teste.pvalue,6)}, maior que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, não se rejeita Ho (o modelo NÃO está MAL especificado)"""
        )

def Teste_J_Davidson_MacKinnon(x1,x2, y, constante = "S", robusta = "N", Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste J para verificar qual o modelo mais adequado (dentre os dois colocados).
    Ho: o modelo 1 é preferível (ver o p-valor do último coeficiente).

    x1: lista ou array com os valores das variáveis independentes do primeiro modelo;
    x2: lista ou array com os valores das variáveis independentes do segundo modelo;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    
    #Fazendo a regressão do segundo modelo
    Regressao_Multipla(x2, y, constante, robusta)
    clear_output()

    #Criando um novo dataframe e adicionando os valores previstos do modelo 2 à x
    Valores_Previstos_2 = pd.DataFrame({'Previsão M1':Lista_ychapeu})
    x = pd.concat([x1, Valores_Previstos_2], axis=1, sort=False)

    #Fazendo a regressão do primeiro modelo sobre x
    Regressao_Multipla(x, y, constante, robusta)
    clear_output()

    #Pegando o p-valor do teste
    P_valor = Resultado.pvalues[-1]

    if P_valor < Nivel_de_Significância:
        print(f"""
        O p-valor do teste foi de {np.around(P_valor,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, rejeita-se Ho (ou seja, o modelo 2 ({list(x2)}) é mais bem especificado)."""
        )
    else:
        print(f"""
        O p-valor do teste foi de {np.around(P_valor,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, não se rejeita Ho (ou seja, o modelo 1 ({list(x1)}) é mais bem especificado)."""
        )

######### Funções de Dados em Painel #########
def Arrumar_Painel():
    '''
    Função que transforma o painel num formato que o PanelOLS consegue ler (index multinível e coluna do tipo categoria para os anos)
    '''
    global df

    # pedir a coluna com os indivíduos; se o nome for inválido, sair da função.
    coluna_individuos = str(input('Qual o rótulo da coluna de indivíduos/observações?\n'))
    if coluna_individuos not in df.columns:
        print("Coluna de indivíduos não está no dataframe. Insira uma coluna válida e tente novamente!")
        return None
    
    # pedir a coluna com os períodos de tempo; se o valor for inválido, sair da função.
    coluna_tempo = str(input('Qual o rótulo da coluna de tempo?\n'))
    if coluna_tempo not in df.columns:
        print("Coluna de tempo não está no dataframe. Insira uma coluna válida e tente novamente!")
        return None

    ## arrumando o painel
    periodos = pd.Categorical(df[coluna_tempo])
    df = df.set_index([coluna_individuos,coluna_tempo])
    df[coluna_tempo] = periodos
    return df

def Reg_Painel_Primeiras_Diferenças (x,y, robusta = "N"):
    '''
    Função que calcula uma regressão de primeiras diferenças SEM um intercepto, sendo, por default, computada com erros padrões não robustos.
    Para calcular a regressão com um intercepto, ver o notebook "Cap 13 e 14".
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''
    Modelo = FirstDifferenceOLS(y, x)
    Resultado = Modelo.fit()
    print(Resultado)

def Reg_Painel_Efeitos_Fixos(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão de efeitos fixos, sendo, por default, computada com um intercepto e com erros padrões não robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = PanelOLS(y,X, entity_effects=True)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'robust')
    print(Resultado)

def Reg_Painel_MQO_Agrupado(x, y, constante = "S", robusta = "S"):
    '''
    Função que calcula uma regressão por MQO agrupado, sendo, por default, computada com um intercepto e com erros padrões  robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão robustos.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = PooledOLS(y,X)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'robust')
    print(Resultado)

def Reg_Painel_Efeitos_Aleatórios(x, y, constante = "S"):
    '''
    Função que calcula uma regressão de efeitos fixos, sendo, por default, computada com um intercepto e com erros padrões  robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão robustos.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo
    Modelo = RandomEffects(y,X)
    Resultado = Modelo.fit()
    print(Resultado)

In [4]:
coletar_dados('jtrain')

jtrain.dta foi lido com sucesso!


,year,fcode,employ,sales,avgsal,scrap,rework,tothrs,union,grant,...,grant_1,clscrap,cgrant,clemploy,clsales,lavgsal,clavgsal,cgrant_1,chrsemp,clhrsemp
0,1987,410032.0,100.0,47000000.0,35000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,10.463103,NaN,NaN,NaN,NaN
1,1988,410032.0,131.0,43000000.0,37000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,0.270027,-0.088949,10.518673,0.055570,0.0,-8.946565,-1.165385
2,1989,410032.0,123.0,49000000.0,39000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,-0.063013,0.130621,10.571317,0.052644,0.0,0.198597,0.047832
3,1987,410440.0,12.0,1560000.0,10500.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,9.259130,NaN,NaN,NaN,NaN
4,1988,410440.0,13.0,1970000.0,11000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,0.080043,0.233347,9.305651,0.046520,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,1988,419483.0,108.0,11500000.0,14810.0,25.0,NaN,0.0,1,0,...,0,0.223144,0,-0.208218,0.044453,9.603058,0.059321,0.0,0.000000,0.000000
467,1989,419483.0,129.0,12000000.0,14227.0,30.0,NaN,20.0,1,0,...,0,0.182322,0,0.177681,0.042559,9.562897,-0.040161,0.0,3.100775,1.411176
468,1987,419486.0,80.0,7000000.0,16000.0,NaN,NaN,0.0,0,0,...,0,NaN,0,NaN,NaN,9.680344,NaN,NaN,NaN,NaN
469,1988,419486.0,90.0,8500000.0,17000.0,NaN,NaN,0.0,0,0,...,0,NaN,0,0.117783,0.194157,9.740969,0.060625,0.0,0.000000,0.000000


In [5]:
df.columns

Index(['year', 'fcode', 'employ', 'sales', 'avgsal', 'scrap', 'rework',
       'tothrs', 'union', 'grant', 'd89', 'd88', 'totrain', 'hrsemp', 'lscrap',
       'lemploy', 'lsales', 'lrework', 'lhrsemp', 'lscrap_1', 'grant_1',
       'clscrap', 'cgrant', 'clemploy', 'clsales', 'lavgsal', 'clavgsal',
       'cgrant_1', 'chrsemp', 'clhrsemp'],
      dtype='object')

In [6]:
Arrumar_Painel()

employ       sales   avgsal  scrap  rework  tothrs  union  \
fcode    year                                                              
410032.0 1987   100.0  47000000.0  35000.0    NaN     NaN    12.0      0   
         1988   131.0  43000000.0  37000.0    NaN     NaN     8.0      0   
         1989   123.0  49000000.0  39000.0    NaN     NaN     8.0      0   
410440.0 1987    12.0   1560000.0  10500.0    NaN     NaN    12.0      0   
         1988    13.0   1970000.0  11000.0    NaN     NaN    12.0      0   
...               ...         ...      ...    ...     ...     ...    ...   
419483.0 1988   108.0  11500000.0  14810.0   25.0     NaN     0.0      1   
         1989   129.0  12000000.0  14227.0   30.0     NaN    20.0      1   
419486.0 1987    80.0   7000000.0  16000.0    NaN     NaN     0.0      0   
         1988    90.0   8500000.0  17000.0    NaN     NaN     0.0      0   
         1989   100.0   9900000.0  18000.0    NaN     NaN    40.0      0   

               grant  d89  d88  ...   clscrap  cgrant  clemploy   clsales  \
fcode    year                   ...                                         
410032.0 1987      0    0    0  ...       NaN       0       NaN       NaN   
         1988      0    0    1  ...       NaN       0  0.270027 -0.088949   
         1989      0    1    0  ...       NaN       0 -0.063013  0.130621   
410440.0 1987      0    0    0  ...       NaN       0       NaN       NaN   
         1988      0    0    1  ...       NaN       0  0.080043  0.233347   
...              ...  ...  ...  ...       ...     ...       ...       ...   
419483.0 1988      0    0    1  ...  0.223144       0 -0.208218  0.044453   
         1989      0    1    0  ...  0.182322       0  0.177681  0.042559   
419486.0 1987      0    0    0  ...       NaN       0       NaN       NaN   
         1988      0    0    1  ...       NaN       0  0.117783  0.194157   
         1989      1    1    0  ...       NaN       1  0.105361  0.152469   

                 lavgsal  clavgsal  cgrant_1    chrsemp  clhrsemp  year  
fcode    year                                                            
410032.0 1987  10.463103       NaN       NaN        NaN       NaN  1987  
         1988  10.518673  0.055570       0.0  -8.946565 -1.165385  1988  
         1989  10.571317  0.052644       0.0   0.198597  0.047832  1989  
410440.0 1987   9.259130       NaN       NaN        NaN       NaN  1987  
         1988   9.305651  0.046520       0.0   0.000000  0.000000  1988  
...                  ...       ...       ...        ...       ...   ...  
419483.0 1988   9.603058  0.059321       0.0   0.000000  0.000000  1988  
         1989   9.562897 -0.040161       0.0   3.100775  1.411176  1989  
419486.0 1987   9.680344       NaN       NaN        NaN       NaN  1987  
         1988   9.740969  0.060625       0.0   0.000000  0.000000  1988  
         1989   9.798127  0.057158       0.0  36.000000  3.610918  1989  

[471 rows x 29 columns]

In [10]:
df.describe()

,employ,sales,avgsal,scrap,rework,tothrs,union,grant,d89,d88,...,grant_1,clscrap,cgrant,clemploy,clsales,lavgsal,clavgsal,cgrant_1,chrsemp,clhrsemp
count,440.000000,373.0,406.000000,162.000000,123.000000,415.000000,471.000000,471.000000,471.000000,471.000000,...,471.000000,108.000000,471.000000,290.000000,245.000000,406.000000,267.000000,314.000000,251.000000,251.000000
mean,59.315909,6116036.5,18872.824219,3.843642,3.473984,29.200000,0.197452,0.140127,0.333333,0.333333,...,0.076433,-0.221132,0.063694,0.082022,0.115873,9.784861,0.060258,0.114650,5.935913,0.503702
std,74.123782,7912517.5,6703.156250,6.007770,5.462483,47.699755,0.398500,0.347488,0.471906,0.471906,...,0.265972,0.579248,0.461471,0.253524,0.438901,0.350157,0.076471,0.319107,26.618122,1.604679
min,4.000000,110000.0,4237.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-3.314186,-1.000000,-0.980829,-1.982874,8.351611,-0.405465,0.000000,-88.622551,-4.025352
25%,15.000000,1550000.0,14102.500000,0.592500,0.350000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.397529,0.000000,-0.028988,-0.011011,9.554108,0.022276,0.000000,-0.072566,-0.014926
50%,30.000000,3000000.0,17773.000000,1.415000,1.160000,12.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.141095,0.000000,0.070657,0.107108,9.785427,0.057158,0.000000,0.198597,0.034786
75%,72.000000,7700000.0,22360.000000,4.000000,4.000000,40.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,0.009295,0.000000,0.182322,0.223144,10.015029,0.090763,0.000000,11.009523,1.368111
max,525.000000,54000000.0,42583.000000,30.000000,40.000000,320.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,2.397895,1.000000,1.673976,2.896701,10.659210,0.568912,1.000000,142.000000,4.394449


In [7]:
x = df[['d88','d89','union','grant','grant_1']]
y = df['lscrap']
Reg_Painel_MQO_Agrupado(x,y,robusta='S')

                          PooledOLS Estimation Summary                          
Dep. Variable:                 lscrap   R-squared:                        0.0465
Estimator:                  PooledOLS   R-squared (Between):              0.0382
No. Observations:                 162   R-squared (Within):               0.1297
Date:                Thu, Mar 11 2021   R-squared (Overall):              0.0465
Time:                        08:58:07   Log-likelihood                   -289.72
Cov. Estimator:                Robust                                           
                                        F-statistic:                      1.5230
Entities:                          54   P-value                           0.1856
Avg Obs:                       3.0000   Distribution:                   F(5,156)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             1.6333
                            

In [11]:
x = df[['d88','d89','grant','grant_1']]
Reg_Painel_Efeitos_Fixos(x,y)
fixed = Resultado

                          PanelOLS Estimation Summary                           
Dep. Variable:                 lscrap   R-squared:                        0.2010
Estimator:                   PanelOLS   R-squared (Between):             -0.0177
No. Observations:                 162   R-squared (Within):               0.2010
Date:                Thu, Mar 11 2021   R-squared (Overall):              0.0021
Time:                        09:04:47   Log-likelihood                   -80.946
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      6.5426
Entities:                          54   P-value                           0.0001
Avg Obs:                       3.0000   Distribution:                   F(4,104)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             6.5426
                            

In [12]:
Reg_Painel_Efeitos_Aleatórios(x,y)
random = Resultado

                        RandomEffects Estimation Summary                        
Dep. Variable:                 lscrap   R-squared:                        0.1406
Estimator:              RandomEffects   R-squared (Between):             -0.0149
No. Observations:                 162   R-squared (Within):               0.2005
Date:                Thu, Mar 11 2021   R-squared (Overall):              0.0046
Time:                        09:05:06   Log-likelihood                   -113.25
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      6.4215
Entities:                          54   P-value                           0.0001
Avg Obs:                       3.0000   Distribution:                   F(4,157)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             6.4215
                            

In [14]:
# Hausman test for fixed versus random effects model
# null hypothesis: random effects model
psi = fixed.cov - random.cov
diff = fixed.params - random.params
# psi = fixed1.cov.iloc[1:,1:] - random.cov.iloc[1:,1:]
# diff = fixed1.params[1:] - random.params[1:]
W = diff.dot(np.linalg.inv(psi)).dot(diff)
dof = random.params.size -1
pvalue = stats.chi2(dof).sf(W)
print("Hausman Test: chisq = {0}, df = {1}, p-value = {2}".format(W, dof, pvalue))

Hausman Test: chisq = 1.1477301692219588, df = 4, p-value = 0.8866272220479142


In [17]:
def hausman_EF_EA(x_inef, y, Nivel_de_Significância = 0.05):
    '''
    Função que faz um teste de Hausman, em que H0: Não há correlação entre os efeitos não-observados e as variáveis explicativas
    x_inef: variáveis explicativas do modelo ineficiente sob H0 (EF);
    y: variável explicativa
    '''
    ## Fazendo a regressão de efeitos fixos e guardando o resultado
    Reg_Painel_Efeitos_Fixos(x,y)
    clear_output()
    fixed = Resultado

    ## Fazendo a regressão de efeitos aleatórios e guardando o resultado
    Reg_Painel_Efeitos_Aleatórios(x,y)
    clear_output()
    random = Resultado

    ## Calculando a estatística de Hausman
    # calculando a diferença entre os parametros e a variância assíntótica da diferença entre os parametros
    var_assin = fixed.cov - random.cov
    d = fixed.params - random.params
    
    # calculando H
    H = d.dot(np.linalg.inv(var_assin)).dot(d)

    # calculando os graus de liberdade
    gl = random.params.size -1

    # Calculando o P-valor do teste
    p = stats.chi2(gl).sf(H)

    if p < Nivel_de_Significância:
        print(f"O valor de H é {round(H,6)} com {gl} graus de liberdade na distribuição chi2. O p-valor do teste é {round(p,6)} e, portanto, se rejeita H0 e prefere-se o modelo de efeitos fixos.")
    else:
        print(f"O valor de H é {round(H,6)} com {gl} graus de liberdade na distribuição chi2. O p-valor do teste é {round(p,6)} e, portanto, não se rejeita H0 e prefere-se o modelo de efeitos aleatórios.")

In [18]:
hausman_EF_EA(x,y)

O valor de H é 1.14773 com 4 graus de liberdade. O p-valor do teste é 0.886627 e, portanto, não se rejeita H0 e prefere-se o modelo de efeitos aleatórios.
